- author: Lee Meng
- date: 2018-03-13 16:34
- title: Google Data Studio 基礎
- slug: google-data-studio-basics
- tags: 資料科學, data-science, 資料視覺化
- description: Google Data Studio 是 Google 推出的一個儀表板服務，讓我們可以利用多種連結器將儲存在如 Google Analytics、 Google 試算表及 Google BigQuery 等特定資料來源的資料做出漂亮的 visualization ，用資料講故事而不用自己設計 UI。這篇把學到的一些技巧以及使用心得記錄下來。
- summary: Google Data Studio 是 Google 推出的一個儀表板服務，讓我們可以利用多種連結器將儲存在如 Google Analytics、 Google 試算表及 Google BigQuery 等特定資料來源的資料做出漂亮的 visualization ，用資料講故事而不用自己設計 UI。這篇把學到的一些技巧以及使用心得記錄下來。
- image: jonathan-velasquez-160775-unsplash.jpg
- image_credit_url: https://unsplash.com/photos/c1ZN57GfDB0?utm_source=unsplash&utm_medium=referral&utm_content=creditCopyText

[Google Data Studio](https://cloud.google.com/data-studio/) 是 Google 推出的一個 Dashboard / Reporting 的服務，讓我們可以利用多種[連結器](https://support.google.com/datastudio/answer/7530149?hl=en&ref_topic=6370347)將儲存在如 Google Analytics、 Google 試算表及 Google BigQuery 等特定資料來源的資料做出漂亮的 visualization ，用資料講故事而不用自己設計 UI。公司內部雖然有自己的 dashboards 不過想說多試一些方案沒有壞處，而且現在 Data Studio 還是 Beta 版本，雖然介面是中文，說明文件還只有英文，想說把學到的一些技巧以及使用心得記錄下來。

## 將 Google 試算表的資料可視化

為了快速展示 Data Studio 的功能，我們將使用[政府資料開放平臺](https://data.gov.tw/)上由交通部觀光局提供的[105年來台旅客性別統計](https://data.gov.tw/dataset/45444)資料。將 CSV 檔案下載下來，稍微簡化格式後上傳到 Google 試算表以當作報表的資料來源。下圖是簡化後的資料：

<center>
    <img src="images/google-data-studio/table-preview.png" style=""/>
</center>
<center>
    <a href="https://data.gov.tw/dataset/45444" target="_blank">資料來源</a>
    <font color='purple'>: 2016年來台旅客性別統計</font><br>
    每一列代表某地區 / 國家的訪台人數以及男女比
</center>

## 條件欄位應用
[條件欄位](https://support.google.com/datastudio/answer/7359285)讓我們可以針對試算表裡頭每一列做 IF ELSE 判斷，依照判斷結果給予不同的值。現在假設我們想知道有多少國家的男性遊客過半數，可以使用簡易的評量表來計算：

<center>
    <img src="images/google-data-studio/male-over-half.png"/>
</center>
<center>
    <font color='purple'>訪台男性遊客過半國家佔全部國家的比例<br>
</center>

我們發現高達八成的國家（有些是區域）的訪台男性遊客較女性為多。我們可以調查其他國家的訪客性別比，看是不是只有台灣有此現象。要產生分母的「國家數」很直覺，我們只要新增一個欄位並計算有幾個國家即可：

<center>
    <img src="images/google-data-studio/num_countries_as_metric.png"/>
</center>
<center>
    <font color='purple'>新增一個名為「國家數」的欄位<br>
</center>

但要計算分子的「男性遊客過半國家數」就稍微 tricky 了。我們想做的是，針對每一國家（每一列），只有在該國訪台男性遊客百分比過半（超過 50%)的時候才會被納入結果。而 Data Studio 的[條件欄位](https://support.google.com/datastudio/answer/7359285)就是專門針對這種情況設計的。



<center>
    <img src="images/google-data-studio/conditional-calculated-field.png"style="width:70%"/>
</center>
<center>
    <font color='purple'>使用 CASE 語法對每一列做 IF-ELSE 判斷<br>
</center>

上面的公式用白話來說就是：

In [ ]:
#blockquote
針對每一列的國家，看它的「男性百分比」欄位的值有沒有大於50。有的話值為1，否則為0。
在針對每列做完條件判斷以後再把所有 1 加起來，就等於符合條件的國家數。

## 篩選器（filter）應用

根據上個分析，我們知道女性遊客過半的國家只佔 20%。假設我們想確切知道是哪些國家的女性遊客過半，可以從女性百分比最高的國家開始列出男女比：

<center>
    <img src="images/google-data-studio/multibar-female-over-half.png" style="width:100%"/>
</center>
<center>
    <font color='purple'>訪台女性遊客過半國家<br>
</center>

我們發現女性遊客過半的都是亞洲國家，或許我們可以簡單解釋成這些國家與台灣的距離短，適合女性遊客拜訪。而為了讓圖表易讀，上面這張組合圖額外建立一個篩選器來過濾掉男性遊客比女性多的國家：

<center>
    <img src="images/google-data-studio/female-more-than-half.png" style="width:70%"/>
</center>
<center>
    <font color='purple'>新增一個篩選器以過濾男性遊客比例較高的國家<br></font>
    註：一般的長條圖可以直接透過設定限制長條圖數目
</center>




## 維度 VS 指標
在 Data Studio 裡頭，了解[維度跟指標的差異](https://support.google.com/datastudio/answer/6402048?ref_topic=7441655&utm_source=product&utm_medium=cta&utm_campaign=wwr&utm_content=dims_mets)很重要。

以我們現在的資料集為例，每一列就是一筆紀錄（record），每一行則是一個欄位。每個欄位則是維度或指標。

- 指標（Metric，底下藍色）
    * 數值型欄位，有經過「匯總」，負責 quantify 資料
    * 如「國家數」、「總人數」
- 維度（Dimension，底下綠色）
    * 分類型欄位，負責 qualify 資料
    * 如「國家」、「居住地」

<center>
    <img src="images/google-data-studio/metric-versus-dimension.png" style=""/>
</center>
<center>
    <font color='purple'>fx 則代表是額外利用公式建立的欄位</font><br>
</center>

像我們前面定義的「男性遊客過半國家數」欄位因為有經過 `SUM` 公式匯總成為一個數值，因此為一個指標（藍）。而如果我們透過 `CASE` 語法新定義一個「男性過半」欄位如下：

<center>
    <img src="images/google-data-studio/define-a-dimension-with-case-syntax.png" style="width:70%"/>
</center>

此欄位沒有經過匯總因此被視為維度，在上一張圖被標為綠色。因此一句話總結維度跟指標的功能就是：

In [ ]:
#blockquote
維度負責「描述」資料； 指標則負責「衡量」資料。

## 資料透視表 (Pivot Table)
資料透視表很適合拿來看在不同條件下某個指標的表現。下圖是一個依照

- 居住地
- 國家

兩個維度計算「男性人數」指標的資料透視表：

<center>
    <img src="images/google-data-studio/pivot-table.png" style=""/>
</center>

依照[官方文件](https://support.google.com/datastudio/answer/7516660?hl=en)有幾點值得注意：
- 資料透視表最多處理 50,000 筆資料，為了避免 scan 資料太花時間，可以額外建立一些篩選器 subset 資料
- 列維度跟欄維度最多可以分別設定 2 個維度（上例列欄各設定 1 個維度）

## 限制
可能因為還處在 beta 版本，在這篇文章寫的時候（2018/03）試用了一陣子發現 Data Studio 也有一些使用案例沒有辦法做到，像是：
- 篩選器（filter）只能設定像是「欄位 C 大於 X」這種條件，而不能做「當欄位 C1 > 欄位 C2」這種欄位間的比較。
- 同上，條件欄位也只能設定像是「欄位 C 大於某固定值 X」的條件
- 資料透視表包含的資料稍多 (> 2000筆)就開始變慢 ..



## 實戰演練
這篇文章用的報表連結在[此](https://datastudio.google.com/open/19a--FnAQ9asll18anhv7negoXGO0fpyH)，可以自己試試不同 visualization。有任何 feedback 也歡迎聯絡。

<center>
    <img src="images/google-data-studio/google-data-studio-preview.png" style="width:70%"/>
</center>

In [ ]:
#ignore
## TODO

- Google Sheet
    * 目前手動做法是上傳 csv 到 雲端硬碟(data sources 資料夾) -> 建立新資料來源 -> Google 試算表
- [Connector](https://support.google.com/datastudio/topic/6370347?ref_topic=7441382&utm_source=product&utm_medium=cta&utm_campaign=wwr&utm_content=connectors)
    * Connector 是 目前看的 data 跟 data source 之間的 pipe


## 圖表類型
- Bullet chart 可以用來表示指標有沒有達標

## 條件判斷

維度 vs 指標
https://support.google.com/datastudio/answer/6402048?ref_topic=7441655&utm_source=product&utm_medium=cta&utm_campaign=wwr&utm_content=dims_mets


一般來說 viewer 在 view mode 不能改變 filter, 要讓他們改變要另外加一個 date range control
- [Learn more about working with dates](https://support.google.com/datastudio/answer/6401549?utm_source=product&utm_medium=cta&utm_campaign=wwr&utm_content=daterange_prop)

- [Learn more about filtering your data](https://support.google.com/datastudio/answer/6291066?ref_topic=7327560&utm_source=product&utm_medium=cta&utm_campaign=wwr&utm_content=filter_prop)

- [Chart 種類](https://support.google.com/datastudio/topic/7059081?ref_topic=6371617&utm_source=product&utm_medium=cta&utm_content=explore_charts&utm_campaign=wwr&utm_content=explore_charts)

- [Video tutorials](https://support.google.com/datastudio/answer/6390659?ref_topic=7450642&utm_source=product&utm_medium=cta&utm_campaign=wwr&utm_content=next_steps)


- [自定義第三方 connectors](https://developers.google.com/datastudio/connector/?utm_source=product&utm_medium=cta&utm_campaign=wwr&utm_content=next_steps)


- 可以輸入google analytics Track ID （地8頁）
    * 檔案 -> 報表設定的右邊
    * 之後要看該 report 的 GA結果有兩種方式
        * 在GA看 
        * 在 data studio 把 ga view 以 data source 輸入，查看
        
- 客製化, 讓人可以切換到他們自己的 account 來看結果（主要應用：google analytics?)
- [Report Templates](https://support.google.com/datastudio/answer/6292498?hl=en&ref_topic=6267740)
    * You can append /preview to the end of the report's URL to add a USE TEMPLATE button to the report. This button makes your report behave exactly like the sample reports built into Data Studio. This makes it even easier for you to create your own report templates.



## TODO
把 marketing 的資料放入 data studio

Graphql
Stream data: Kafka
airflow
data science artechiture